## NEXT:
- Clip params in softmax parameterization?

In [ ]:
import gym_examples
import numpy as np
import gym

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.size'] = 12
rcParams['figure.figsize'] = (10, 4)

In [ ]:
def action2str(action):
    left = u'\u2190'
    right = u'\u2192'
    up = u'\u2191'
    down = u'\u2193'
    policy_str = ""
    if 0 == action:
        policy_str += down 
    if 1 == action:
        policy_str += right 
    if 2 == action:
        policy_str += up
    if 3 == action:
        policy_str += left
    return policy_str

def getAngle(arr):
    down, right, up, left = arr
    x = right-left
    y = up-down
    if x > 0 and y >= 0: return np.arctan(y/x) /np.pi*180
    if x < 0 and y >= 0: return 180 - np.arctan(y/abs(x)) /np.pi*180
    if x > 0 and y < 0: return 360 - np.arctan(abs(y)/x) /np.pi*180
    if x < 0 and y < 0: return 180 + np.arctan(abs(y)/abs(x)) /np.pi*180

In [ ]:
# Policy is a map from the states to a distribution over actions
# We assume the same actions can be taken from each state and they are
# indexed by integers starting at 0
"""
    size: size of the grid of the environment
    A: number of actions
    S: number of states (not currently used)
"""
class Policy:
    def __init__(self, size, A=0, S=0):
        # Initialise as uniform distribution
        self.S = S
        self.A = A
        self.size = 3
        self.distribution = np.ones((3, 3, A))/A
    
    # sample an action according to the policy in the given state 
    def __call__(self, state):
        return np.random.choice(np.arange(self.A), p=self.distribution[state])
    
    def plotPolicy(self):
        #size = int(np.sqrt(self.S))
        plt.imshow(np.zeros((self.size, self.size)))
        #actions = np.argmax(self.distribution, axis=1)
        for i in range(self.size):
            for j in range(self.size):
                angle = getAngle(self.distribution[i,j])
                plt.text(j, i, u'\u2192', ha="center", va="center", color="r", fontsize=24,
                        rotation=angle, rotation_mode='anchor')
    
class SoftmaxPolicy(Policy):
    def __init__(self, size, A, S=0):
        super().__init__(size, A, S)
        self.params = np.ones((self.size, self.size, self.A))
        
        
    def __call__(self, state):
        p = np.exp(self.params[state]) / np.sum(np.exp(self.params[state]))
        return np.random.choice(np.arange(self.A), p = p)
    
    def plotPolicy(self):
        self.updateDistribution()
        plt.imshow(np.zeros((self.size, self.size)))
        for i in range(self.size):
            for j in range(self.size):
                angle = getAngle(self.distribution[i,j])
                plt.text(j, i, u'\u2192', ha="center", va="center", color="r", fontsize=24,
                        rotation=angle, rotation_mode='anchor')
    
    def normaliseParams(self):
        self.params = self.params - np.repeat(np.reshape(np.mean(self.params, axis=2), (self.size, self.size, 1)), 
                                              self.A, axis=2)
    
    def updateDistribution(self):
        for i in range(self.size):
            for j in range(self.size):
                self.distribution[i,j] = np.exp(self.params[i,j]) / np.sum(np.exp(self.params[i,j]))
        
    
# Have to "select" a policy at the start of each episode
class MixturePolicy():
    def __init__(self, alpha, C):
        self.alpha = alpha
        self.C = C
        self.activePolicy = None
        
    def selectPolicy(self):
        self.activePolicy = np.random.choice(self.C, p=self.alpha)   
        
    def __call__(self, state):
        return self.activePolicy(state)
    
    def plotPolicy(self):
        for pol in self.C:
            pol.plotPolicy()
    


In [ ]:
"""
    env: the CMDP environment
    policy: mixture policy to compute the state distribution of
    eps0: error tolerance
    M: number of episodes
    t0: number of time steps per episode
"""
def DensityOracle(env, mix_policy, eps, eps0, M=10, t0=1000):
    
    #env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=3,
    #               rewards=np.array([[1,2,3],[4,5,6],[7,8,9]]), costs=np.array([[1,2,3],[4,5,6],[7,8,9]]))
    
    delta = 0.05
    gamma = 0.9
    #M = int(200/(eps0**2) * np.log(2*S*np.log(0.1*eps)/(delta*np.log(gamma))))
    #t0 = int(np.log(0.1*eps0)/np.log(gamma))
    print(M, t0)
    
    state_frequencies = np.zeros((M, env.size, env.size))
    
    
    for m in range(M):
        observation = env.reset()[0]['agent']
        state = (observation[0], observation[1])
        cur_gamma = 1
        mix_policy.selectPolicy()
        for t in range(t0):
            
            action = mix_policy(state)  # agent policy that uses the observation and info
            obs = env.step(action)[0]['agent']
            state_frequencies[m, obs[0], obs[1]] += cur_gamma
            cur_gamma *= gamma
            

    env.close()
    return np.sum(state_frequencies, axis=0)/M*(1-gamma)/(1-np.exp(np.log(gamma)*t0))


"""
    env: the CMDP environment
    reward: vector of rewards
    policy: policy
    eps1: error tolerance (should there be two eps here for reward
    and constraint violations?)
    T: number of iterations
    K: number of episodes per iteration
    b: utility threshold
"""
def MinMaxOracle(env, policy, reward, eps1, T=10, K=10,  b=5, lr_factor=1, lamb_boost = 10):
    
    # Normalising the reward (could also normalise with max)
    rewards = reward/np.sum(reward)
    print('Rewards: \n', rewards)
    env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=env.size,
                  rewards=rewards, costs=env.costs)
    
    # Initialise policy as uniform
    policy = SoftmaxPolicy(env.size, env.A) # <====
    
    gamma = 0.9 # <====
    
    # Chosen according to theorem 1
    eta1 = 2*np.log(env.A)/lr_factor
    eta2 = (1-gamma)/np.sqrt(T)/lr_factor*lamb_boost
    xi = 1/8 # <====
    
    lamb = 0; 
    for t in range(T):
        VL = np.zeros((env.size, env.size)); QL = np.zeros((env.size, env.size, env.A))
        AL = np.zeros((env.size, env.size, env.A))
        
        for k in range(K):
            # Sampling (s,a) from initial state-action space (is action_space.sample() fine here? Different actions in different
            # states may prove problematic)
            
            K_ = np.random.geometric(1-gamma) # Use the same or different K_ for Q_hat and V_hat? # <====
            for i in range(env.size):
                for j in range(env.size):
                    # Setting the environment to the correct state (Is this a restrictive assumption??) # <========
                    env.reset(state=np.array([i,j]))

                    # Estimating V_hat
                    state = (i,j)
                    for k_ in range(K_): 
                        action = policy(state)
                        observation, reward_cost, terminated, truncated, info = env.step(action)
                        state = (observation['agent'][0], observation['agent'][1])
                        
                        VL[i,j] += reward_cost[0] + lamb*reward_cost[1]


                    for a in range(env.A):
                        # Estimating Q_hat
                        env.reset(state=np.array([i,j]))
                        action = a
                        for k_ in range(K_):
                            observation, reward_cost, terminated, truncated, info = env.step(action)
                            state = (observation['agent'][0], observation['agent'][1])
                            action = policy(state)

                            QL[i,j,a] += reward_cost[0] + lamb*reward_cost[1]

            Vg_hat_rho = 0
            #for k in range(K):
            observation, info = env.reset()
            state = (observation['agent'][0], observation['agent'][1])
            #K_ = np.random.geometric(1-gamma)
            #Vg_hat_s = 0
            for k_ in range(K_): 
                action = policy(state)
                observation, reward_cost, terminated, truncated, info = env.step(action)
                state = (observation['agent'][0], observation['agent'][1])
                #Vg_hat_s += reward_cost[1]
                Vg_hat_rho += reward_cost[1]
            #Vg_hat_rho += Vg_hat_s/K
        Vg_hat_rho = Vg_hat_rho / K
            
        # This should be the same as the way they do it
        VL = VL / K # <====
        QL = QL / K
             
        AL = QL - np.repeat(np.reshape(VL, (env.size, env.size,1)), env.A, axis=2)
        
        
        
        policy.params += eta1/(1-gamma)*AL
        
        
        policy.normaliseParams()
        #policy.updateDistribution()
        #print(policy.distribution)
        
        #policy.plotPolicy()
        #plt.show()
        #min(1/((1-gamma)*xi), lamb - eta2*(Vg_hat_rho - b))
        lamb = max(0, min(1/((1-gamma)*xi), lamb - eta2*(Vg_hat_rho - b)))
        #print(Vg_hat_rho, lamb - eta2*(Vg_hat_rho - b), lamb)
        #print(Vg_hat_rho)
        #print(lamb)
        
    policy.plotPolicy()
    plt.show()
    
    return policy

In [ ]:
# ALGORITHM 1
"""
    env: the CMDP environment 
    R: the (convex) objective in the state distribution
    dR: gradient of the (convex) objective for ease of computation
    eta: step size
    T: number of iterations
    K: the constraint limit (should this be here on in the environment?)
    eps0: state distribution oracle error tolerance
    eps1: MinMax Oracle error tolerance
"""
def algorithm1(env, dR, eps, eta, eps0, eps1, b=0, T=10, Tmm=20, K=20):
    init_policy = SoftmaxPolicy(env.size, env.A)
    init_policy.params = np.array([
                          [[0.9, 0.05, 0.03, 0.02],
                           [0.05, 0.9, 0.03, 0.02],
                           [0.05, 0.03, 0.9, 0.02]],
        
                          [[0.02, 0.05, 0.03, 0.9],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025]],
        
                          [[0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025]]
                        ])
    init_policy.updateDistribution()
    init_policy.plotPolicy()
    plt.show()
    C = [init_policy] # arbitrary policy
    alpha = np.array([1])
    for t in range(T):
    
        mixture_policy = MixturePolicy(alpha, C)
        d = DensityOracle(env, mixture_policy, eps, eps0, M=1000, t0=100)
        new_policy = MinMaxOracle(env, 0, dR(d),  eps1, T=Tmm, K=K,  b=b, lr_factor=1)
        
        C.append(new_policy)
        alpha = np.append((1-eta)*alpha, eta)
        
    return MixturePolicy(alpha, C)

In [ ]:
env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=3, 
               rewards=np.array([[1,2,3],[4,5,6],[7,8,9]]), costs=np.array([[1,1,1],[1,1,1],[1,1,1]])/9)

### MinMax Oracle 

In [ ]:
# np.array([[1,2,1],[1,5,1],[1,3,1]])
policy = SoftmaxPolicy(3, 4)
policy = MinMaxOracle(env, policy, np.array([[1,2,1],
                                                 [1,5,1],
                                                 [1,3,1]]), 0, T=20, K=20, b=1/4, lr_factor=1, lamb_boost=1)

In [ ]:
p = policy
alpha = [1]
C = [policy]
mix_p = MixturePolicy(alpha, C)
res = DensityOracle(env, mix_p, 0,0.1, M=50, t0=100)
res

In [ ]:
eps=5
sigma = 0.1*eps / (2*env.S)
eps0 = 0.1*eps**2/(80*env.S)
eps1 = 0.1*eps
eta = 0.1*eps**2/(40*env.S)
T= 40*env.S*np.log(np.log(env.S)/(0.1*eps))/(0.1*eps**2)

smoothed_max_ent = lambda d: np.sum(-np.log(d+sigma))
smoothed_max_ent_dR = lambda d: -(d/(d+sigma)+np.log(d+sigma))

In [ ]:
gamma = 0.9
delta = 0.05
M = int(200/(eps0**2) * np.log(2*env.S*np.log(0.1*eps)/(delta*np.log(gamma))))
t0 = int(np.log(0.1*eps0)/np.log(gamma))
M, t0, sigma, eps0, eps1, eta, T

In [ ]:
mix_policy = algorithm1(env, smoothed_max_ent_dR, eps=eps, eta=1e-2, eps0=eps0, eps1=eps1, T=200, Tmm=20, K=20)

In [ ]:
res = DensityOracle(env, mix_policy, 0, 0.1, M=5000, t0=100)
res

### Old stuff

In [ ]:
p = Policy(9, 4)
p.distribution = np.array([[0.9, 0.05, 0.03, 0.02],
                           [0.05, 0.9, 0.03, 0.02],
                           [0.05, 0.03, 0.9, 0.02],
                           [0.02, 0.05, 0.03, 0.9],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025],
                           [0.5, 0.45, 0.025, 0.025]])

In [ ]:
env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=3, rewards=np.array([[1,2,3],[4,5,6],[7,8,9]]), costs=np.array([[1,2,3],[4,5,6],[7,8,9]]))

In [ ]:
observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward_cost, terminated, truncated, info = env.step(action)
    print(observation, reward_cost, terminated, truncated, info)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [ ]:
#max_ent_R = lambda d: -np.sum(d * np.log(d))
max_ent_dR = lambda d: -(np.log(d)+1)

In [ ]:
#env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=3,
#                  rewards=np.array([[1,2,3],[4,5,6],[7,8,9]])/45, costs=np.array([[1,1,1],[1,1,1],[1,1,1]])/9)
env = gym.make('gym_examples/GridWorld-v3', render_mode=None, size=3,
                  rewards=np.array([[1,2,3],[4,5,6],[7,8,9]])/49, costs=np.array([[1,0,0],[0,0,0],[0,0,0]]))

In [ ]:
p = policy
p.updateDistribution()
p.distribution, p.params, np.argmax(p.distribution, axis=1)